In [2]:
!pip install seaborn

In [3]:
!pip install tsai

  Obtaining dependency information for tsai from https://files.pythonhosted.org/packages/0c/05/44204ea197bf872279f1d445139283b8d30983fa04d177dc12728a0c7ba8/tsai-0.3.8-py3-none-any.whl.metadata
  Obtaining dependency information for pyts>=0.12.0 from https://files.pythonhosted.org/packages/b3/e3/da2042a20782b105631abe273ca5fef4390e7bdb6f5377c596891262437b/pyts-0.13.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.2/324.2 kB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install adtk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00


In [6]:
!pip install sktime

  Obtaining dependency information for sktime from https://files.pythonhosted.org/packages/e5/81/1141e6eb5a43310f39981d1c61abea9a03a0ecb9d62685cc537fa34786c8/sktime-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for scikit-base<0.7.0 from https://files.pythonhosted.org/packages/14/5f/99cfea6a41a806499f8041dbaf183f0e147fa0a1e51a50a38f195a03b6ef/scikit_base-0.6.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 20.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 12.3 MB/s eta 0:00:00


In [8]:
!pip install statsmodels

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from adtk.detector import QuantileAD, SeasonalAD

from sklearn.model_selection import train_test_split
from sktime.datatypes import convert_to
from sklearn.cluster import DBSCAN, KMeans

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [ ]:
sns.set_style('white')
sns.set(rc={'figure.figsize': (11, 4)})

## 1.

In [ ]:
# dummy dataset
DF_ORIG = pd.read_csv('https://raw.githubusercontent.com/numenta/NAB/master/data/realTweets/Twitter_volume_AMZN.csv', index_col='timestamp', parse_dates=True)
# DF_ORIG = pd.read_csv('https://raw.githubusercontent.com/numenta/NAB/master/data/realTweets/Twitter_volume_IBM.csv', index_col='timestamp', parse_dates=True)

In [ ]:
DF_ORIG.head()

In [ ]:
DF_ORIG.info()

In [ ]:
plt.figure(figsize=(14, 6))
DF_ORIG['value'].plot(linewidth=0.5)
plt.show()

In [ ]:
DF_ORIG['date'] = pd.to_datetime(DF_ORIG.index).date

In [ ]:
DF_ORIG['h'] = pd.to_datetime(DF_ORIG.index).hour

In [ ]:
DF_ORIG.head(1)

In [ ]:
df_date = DF_ORIG.groupby('date').sum()

### TODO: chance that i must drop outliers

In [ ]:
plt.figure(figsize=(14, 6))
df_date['value'].plot()
plt.show()

In [ ]:
df_h = DF_ORIG.groupby(pd.Grouper(freq='1h')).sum(numeric_only=True)
df_h.head(5)

In [ ]:
plt.figure(figsize=(14, 4))
df_h['value'].plot()
plt.show()

In [ ]:
start, end = '2015-02-27', '2015-04-22'

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df_h.loc[start:end, 'value'],marker='.', linestyle='-', linewidth=0.5, label='Hourly')
ax.plot(df_h.resample('D').mean().loc[start:end, 'value'], marker='o',
        markersize=3, linestyle='-', label='Day Mean Resample', color='k')
ax.legend();

In [ ]:
start, end = '2015-03-08', '2015-03-22'

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df_h.loc[start:end, 'value'],marker='.', linestyle='-', linewidth=0.5, label='Hourly')
ax.plot(df_h.resample('D').mean().loc[start:end, 'value'], marker='o', markersize=3, linestyle='-', label='Day Mean Resample', color='k')
ax.legend();

In [ ]:
df_h.h = pd.to_datetime(df_h.index).hour

df_h_h = df_h.groupby('h').mean()

df_h_h['value'].plot(linewidth=0.5)
plt.show()

## 2.

### TODO: simplify imports

In [ ]:
import warnings
import sktime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tools.sm_exceptions import ConvergenceWarning

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    EnsembleForecaster,
    MultiplexForecaster,
    TransformedTargetForecaster,
    make_reduction,
)
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.performance_metrics.forecasting import  MeanAbsolutePercentageError, MeanSquaredError
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.utils.plotting import plot_series
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.tbats import TBATS

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [ ]:
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter('ignore', ConvergenceWarning)

r2_score = lambda y_pred, y_test: 1-np.sum(np.square(y_pred - y_test))/np.sum(np.square(y_test - np.mean(y_test)))

smape = MeanAbsolutePercentageError(symmetric = True)
rmse  = MeanSquaredError(square_root=True)

%matplotlib inline

In [ ]:
df_h_copy = df_h.value.astype('float')

In [ ]:
sktime.utils.plotting.plot_series(df_h_copy)
plt.show()

In [ ]:
X_train, X_test = temporal_train_test_split(df_h_copy, test_size=int(df_h_copy.size * 0.4))

In [ ]:
sktime.utils.plotting.plot_series(X_train, X_test, labels=['y_train', 'y_test'])

In [ ]:
# X_train.to_csv("X_train.csv", index_label="timestamp")
# X_test.to_csv("X_test.csv", index_label="timestamp")
# df_h.to_csv("df_h.csv", index_label="timestamp")

### TODO: change these values

In [ ]:
for i in range(len(X_train)):
    if X_train[i] > 1400:
        X_train[i] = 1400
    if X_train[i] < 250:
        X_train[i] = 250

In [ ]:
FH = ForecastingHorizon(X_test.index, is_relative=False)

In [ ]:
FORECASTER = NaiveForecaster(strategy="mean", sp=24)

In [ ]:
FORECASTER.fit(X_train)

In [ ]:
X_pred = FORECASTER.predict(FH)

In [ ]:
plot_series(X_train, X_test, X_pred, labels=["X_train", "X_test", "X_pred"])

In [ ]:
smape(X_pred.values, X_test.values)

In [ ]:
FORECASTER = NaiveForecaster(strategy="mean", sp=24 * 7)

In [ ]:
FORECASTER.fit(X_train)

In [ ]:
X_pred = FORECASTER.predict(FH)

In [ ]:
plot_series(X_train, X_test, X_pred, labels=["X_train", "X_test", "X_pred"])

In [ ]:
smape(X_pred.values, X_test.values)

In [ ]:
FORECASTER = ThetaForecaster(sp=24 * 7)

In [ ]:
FORECASTER.fit(X_train, fh=FH)

In [ ]:
X_pred = FORECASTER.predict(FH)

In [ ]:
plot_series(X_train, X_test, X_pred, labels=["X_train", "X_test", "X_pred"])

In [ ]:
smape(X_pred.values, X_test.values)

In [ ]:
FORECASTER = ExponentialSmoothing(trend="mul", seasonal="add", sp=24 * 7, method='ls')

In [ ]:
FORECASTER.fit(X_train)

In [ ]:
y_pred = FORECASTER.predict(FH)

In [ ]:
plot_series(X_train, X_test, y_pred, labels=["y_train", "y_test", "y_pred"])

In [ ]:
smape(y_pred.values, X_test.values)

In [ ]:
ses = ExponentialSmoothing()

FORECASTER = TransformedTargetForecaster(
                steps=[
                ("deseasonalize1", Deseasonalizer(model="multiplicative", sp=24)),
                ("deseasonalize2", Deseasonalizer(model="multiplicative", sp=24*7)),
                ("forecaster", ses),
                ])
FORECASTER.fit(X_train)

In [ ]:
X_pred = FORECASTER.predict(FH)

In [ ]:
plot_series(X_train, X_test, X_pred, labels=["y_train", "y_test", "y_pred"])

In [ ]:
smape(y_pred.values, X_test.values)

## 3.

In [ ]:
!pip install pmdarima

In [ ]:
import pmdarima as pm
import statsmodels.api as sm

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sktime.utils.plotting import plot_series
from sktime.forecasting.sarimax import SARIMAX
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split

from sktime.performance_metrics.forecasting import  MeanAbsolutePercentageError, MeanSquaredError
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [ ]:
smape = MeanAbsolutePercentageError(symmetric = True)
rmse  = MeanSquaredError(square_root=True)
r2_score = lambda y_pred, y_test: 1-np.sum(np.square(y_pred - y_test))/np.sum(np.square(y_test - np.mean(y_test)))

In [ ]:
ROLLING_MEAN = X_train.rolling(window = 24).mean()
ROLLING_STD  = X_train.rolling(window = 24).std()

In [ ]:
plt.figure(figsize=(12,4), dpi=100)

plt.plot(X_train - X_train[0],      color = 'blue',  label = 'Original')
plt.plot(ROLLING_MEAN - X_train[0], color = 'red',   label = 'Rolling Mean')
plt.plot(ROLLING_STD,             color = 'black', label = 'Rolling Std')

plt.legend(loc = 'best')
plt.title('Rolling Mean & Rolling Standard Deviation')
plt.show()

In [ ]:
P_THRESHOLD = 0.05

In [ ]:
def check_ADF(y, p_threshold):
    result = adfuller(y)
    adf_value = result[0]
    p_value   = result[1]
    print('ADF Statistic: {:.4f}'.format(adf_value))
    print('p-value: {:.4f}'.format(p_value))
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {:.4f}, {}'.format(key, value, 'outperformed' if adf_value>value else ""))
    print(f'Result: The series is {"not " if p_value < p_threshold else ""}stationary')
    return result

In [ ]:
check_ADF(X_train, P_THRESHOLD);

In [ ]:
X_diff = X_train[:].diff(1).dropna()
rolling_mean = X_diff.rolling(window = 24).mean()
rolling_std  = X_diff.rolling(window = 24).std()

In [ ]:
plt.figure(figsize=(12,4), dpi=100)

plt.plot(X_diff, color = 'blue', label = 'Original')
plt.plot(rolling_mean, color = 'red', label = 'Rolling Mean')
plt.plot(rolling_std, color = 'black', label = 'Rolling Std')

plt.legend(loc = 'best')
plt.title('Rolling Mean & Rolling Standard Deviation')
plt.show()

In [ ]:
result = check_ADF(X_diff, P_THRESHOLD)

In [ ]:
SEASON = 24

In [ ]:
# Plot
fig, axes = plt.subplots(4, 1, figsize=(12,8), dpi=100, sharex=True)

# Original Series
axes[0].plot(X_train[:])
axes[0].set_title('Original Series')
# plt.show()

# Usual Differencing
axes[1].plot(X_train[:].diff(1))
axes[1].set_title('Usual Differencing')

# Seasinal Differencing
axes[2].plot(X_train[:].diff(SEASON))
axes[2].set_title('Seasonal Differencing')

# Seasinal and Usual Differencing
axes[3].plot(X_train[:].diff(1).diff(SEASON))
axes[3].set_title('Usual and Seasonal Differencing')

plt.suptitle('Dataset $CO_2$', fontsize=12)
plt.show()

In [ ]:
X_sdif = X_train[:].diff(1).diff(SEASON).dropna()

In [ ]:
results = check_ADF(X_sdif, P_THRESHOLD);

In [ ]:
plot_acf(X_sdif, title='Autocorrelation of Differenced Series', lags=np.arange(70) )
plt.show()

In [ ]:
plot_pacf(X_sdif, title='Partial Autocorrelation of Differenced Series', method='ywm', lags=np.arange(45));plt.show()
plt.show();

In [ ]:
forecaster = SARIMAX(order= (7, 0, 0), seasonal_order=(3, 1, 0, SEASON))
forecaster.fit(X_train)
print(forecaster.summary())

In [ ]:
forecaster._fitted_forecaster.plot_diagnostics(figsize=(12,8))

In [ ]:
fhin = ForecastingHorizon(X_train.index[1:], is_relative=False)
X_in_samples = forecaster.predict(fhin)

X_out = forecaster.predict(FH)

plot_series(X_train, X_test, X_in_samples, X_out, labels=["X_train", "X_test", "X_in_samples", "X_out"])
plt.show()

In [ ]:
smape(X_out.values, X_test.values)

In [ ]:
model = AutoARIMA(start_p=1,
                  d=1,
                  start_q=0,
                  max_p=5,
                  max_q=5,
                  seasonal=True,
                  start_P=0,
                  start_Q=0,
                  D=1,
                  sp=24,
                  max_order = 5,
                  trace  = True,
                  stepwise = True,
                  n_jobs = 1,
                  error_action='ignore',
                  suppress_warnings=True)

In [ ]:
model.fit(X_train)

In [ ]:
model.summary()

In [ ]:
fhin = ForecastingHorizon(X_train.index[1:], is_relative=False)
X_in_samples = model.predict(fhin)

X_out = model.predict(FH)

plot_series(X_train, X_test, X_in_samples, X_out, labels=["X_train", "X_test", "X_in_samples", "X_out"])

In [ ]:
smape(X_out.values, X_test.values)

## 4.

## 5.

In [ ]:
df = DF_ORIG.copy()

In [ ]:
df['day_of_the_week'] = pd.to_datetime(df.index).weekday
df.sample(5)

In [ ]:
def to_segments(df, column, size = 24):  
    df.index.hour[0]
    start_idx = 24-df.index.hour[0]
    df = df.iloc[start_idx:]
    val = df[[column]].values
    return val[:size*(val.size//size)].reshape(-1,size)

In [ ]:
working_week = to_segments(df[df['day_of_the_week'] <= 5], 'value', size = 24)
week_end = to_segments(df[df['day_of_the_week'] > 5], 'value', size = 24)

In [ ]:
plt.figure()
day = 5
#for i,(c,d) in enumerate(zip([monday, tuesday, wednesday, turhday, friday, saturday, sunday],
#                             ['monday', 'tuesday', 'wednesday', 'turhday', 'friday', 'saturday', 'sunday'])):

for i,(c,d) in enumerate(zip([working_week, week_end],
                             ['working_week', 'week_end'])):
    plt.plot(c[day], label="class " + str(d))
plt.legend(loc="best")
plt.show()
plt.close()

In [ ]:
X = np.concatenate((
                    working_week, 
                    week_end
                   ))
y = np.concatenate((
                    0*np.ones(working_week.shape[0]),
                    1*np.ones(week_end.shape[0]),
                   ))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
df_train = convert_to(np.atleast_3d(X_train).transpose(0,2,1), to_type="nested_univ")
df_test = convert_to(np.atleast_3d(X_test ).transpose(0,2,1), to_type="nested_univ")

In [ ]:
from sktime.classification.interval_based import (TimeSeriesForestClassifier, 
                                                  RandomIntervalSpectralEnsemble, 
                                                  SupervisedTimeSeriesForest)
clf = TimeSeriesForestClassifier(n_estimators=100,random_state=47)
clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(f' test score TimeSeriesForestClassifier {test_score:.3f}')
table = pd.DataFrame([['TimeSeriesForestClassifier', test_score]], columns=['method', 'test_score'])

clf = RandomIntervalSpectralEnsemble(n_estimators=100, random_state=47)
clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(f' test score RandomIntervalSpectralEnsemble {test_score:.3f}')
table2 = pd.DataFrame([['RandomIntervalSpectralEnsemble', test_score]], columns=['method', 'test_score'])
table = pd.concat([table, table2])

clf = SupervisedTimeSeriesForest(n_estimators=100, random_state=47)
clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(f' test score SupervisedTimeSeriesForest {test_score:.3f}')
table2 = pd.DataFrame([['SupervisedTimeSeriesForest', test_score]], columns=['method', 'test_score'])
table = pd.concat([table, table2])

In [ ]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier

clf = ShapeletTransformClassifier(estimator=RandomForestClassifier(n_estimators=100),
                                  n_shapelet_samples=100,
                                  max_shapelets=100,
                                  batch_size=20)

clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(f' test score ShapeletTransformClassifier {test_score:.3f}')
table2 = pd.DataFrame([['ShapeletTransformClassifier', test_score]], columns=['method', 'test_score'])
table = pd.concat([table, table2])

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

clf = KNeighborsTimeSeriesClassifier(n_neighbors=1, distance="ddtw")
clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(f' test score KNeighborsTimeSeriesClassifier {test_score:.3f}')
table2 = pd.DataFrame([['KNeighborsTimeSeriesClassifier', test_score]], columns=['method', 'test_score'])
table = pd.concat([table, table2])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(f' test score RandomForestClassifier {test_score:.3f}')
table2 = pd.DataFrame([['RandomForestClassifier', test_score]], columns=['method', 'test_score'])
table = pd.concat([table, table2])

clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)
print(f' test score KNeighborsClassifier {test_score:.3f}')
table2 = pd.DataFrame([['KNeighborsClassifier', test_score]], columns=['method', 'test_score'])
table = pd.concat([table, table2])

In [ ]:
table

In [ ]:
from tsai.all import *
import warnings

In [ ]:
computer_setup()

In [ ]:
splits = get_splits(y, 
                    n_splits=1, 
                    valid_size=0.3, 
                    test_size=0.1, 
                    shuffle=True, 
                    balance=False, 
                    stratify=True,
                    random_state=42, 
                    show_plot=True, 
                    verbose=True)
splits

In [ ]:
X = np.atleast_3d(X.astype('float')).transpose(0,2,1)
y = y.astype('float')

print(X.shape, y.shape)
print (y)

In [ ]:
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)

bs = 4
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

In [ ]:
archs = [
         (RNNPlus, {'n_layers':3, 'bidirectional': True} ),
         (LSTMPlus,{'n_layers':3, 'bidirectional': True} ),
         (GRUPlus, {'n_layers':3, 'bidirectional': True} ),   
         (RNNPlus, {'n_layers':4, 'bidirectional': True} ),
         (RNNPlus, {'n_layers':4, 'bidirectional': False}),  
         (LSTM,    {'n_layers':3, 'bidirectional': False}), 
         (RNN,     {'n_layers':3, 'bidirectional': True} ), 
         (LSTM,    {'n_layers':3, 'bidirectional': True} ),
         (GRU,     {'n_layers':3, 'bidirectional': True} ),   
         (ResNet, {}), 
         (InceptionTime, {}), 
         (XceptionTime,  {}), 
         (TCN, {}),   
         (LSTM_FCN, {}), 
         (TST, {}),
         (FCN, {}),   
        ]

In [ ]:
from IPython.display import clear_output

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])

for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(20, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, ignore_index=True, inplace=True)
    clear_output()
    display(results)

In [ ]:
arch = LSTM_FCN
k = {}
model = create_model(arch, dls=dls, **k)
model = build_ts_model(arch, arch_config=k, dls=dls)

In [ ]:
learn = ts_learner(dls, arch=model, metrics=accuracy) 
learn.lr_find()

In [ ]:
learn = Learner(dls, model,  metrics=accuracy)
start = time.time()
learn.fit_one_cycle(n_epoch = 100, lr_max = 0.0036, cbs=ShowGraph())

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
probas, _, preds = learn.get_X_preds(X[splits[2]])
from sklearn.metrics import accuracy_score

print(y[splits[2]])
print(probas)

accuracy_score(np.argmax(probas,axis=-1),y[splits[2]])

## 7. Anomalies check

In [ ]:
df = df.drop(['h', 'day_of_the_week'], axis=1)

In [ ]:
quantile_ad = QuantileAD(high=0.99, low=0.01)
anomalies = quantile_ad.fit_detect(df)
anomalies.values.sum()

In [ ]:
df2 = df.copy()
df2['a'] = anomalies['value']
df_a = df2[df2['a'] == True].copy()

In [ ]:
plt.figure(figsize=(14, 6), dpi=80, layout='constrained')
plt.plot(df.value)
plt.plot(df_a.value, color='red', marker='*', linestyle='', label = "anomalies")
plt.show()

In [ ]:
seasonal_vol = SeasonalAD()
anomalies = seasonal_vol.fit_detect(df.value)

In [ ]:
df2 = df.copy()
df2['a'] = anomalies
df_a = df2[df2['a'] == True].copy()

In [ ]:
plt.figure(figsize=(14, 6), dpi=80)
plt.plot(df.value)
plt.plot(df_a.value, color='red', marker='*', linestyle='',)
plt.show()

In [ ]:
y = df.value.values.reshape(-1, 1)

In [ ]:
model=DBSCAN(eps = 100., min_samples = 5)

In [ ]:
model.fit(y)
colors = model.labels_
cls,counts = np.unique(colors, return_counts=True)
sort_idx = np.argsort(counts)

In [ ]:
df2 = df.copy()
df2['a'] = colors
df_a = df2[df2['a'] == -1].copy()

In [ ]:
plt.figure(figsize=(14, 6), dpi=80)
plt.plot(df.value)
plt.plot(df_a.value, color='red', marker='*', linestyle='',)
plt.show()